<a href="https://colab.research.google.com/github/rksab/NLP/blob/main/Fine_tune.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

### Roadmap
* Fine tune a pre-trained model
* Download yelp review dataset
* Train sentiment analyser on it
* We'll use AutoTokenizer and AutoModel from hugging face.





In [ ]:
import os
os.environ["WANDB_DISABLED"] = "true"

In [ ]:
import torch
from torch.optim import AdamW
from transformers import AutoTokenizer, AutoModelForSequenceClassification

# Same as before
checkpoint = "bert-base-uncased"
tokenizer = AutoTokenizer.from_pretrained(checkpoint)
model = AutoModelForSequenceClassification.from_pretrained(checkpoint)
sequences = [
    "I've been waiting for a HuggingFace course my whole life.",
    "This course is amazing!",
]
batch = tokenizer(sequences, padding=True, truncation=True, return_tensors="pt")

# This is new
batch["labels"] = torch.tensor([1, 1])

optimizer = AdamW(model.parameters())
loss = model(**batch).loss
loss.backward()
optimizer.step()

In [ ]:
from datasets import load_dataset


In [ ]:
!pip install -U datasets

In [ ]:
#load dataset
dataset = load_dataset("yelp_review_full")

In [ ]:
dataset

In [ ]:
dataset['train'] = dataset['train'].select(range(12000))
dataset['test'] = dataset['test'].select(range(1000))

In [ ]:
dataset['train'][:5]

In [ ]:
dataset['train'].features

we've ratings from 1-5. Let's add a function which adds sentiment to the dataset.

In [ ]:
# change ratings to 3 sentiments: negative: 0, positive: 1 and neutral: 2
def transform(sentence):
  rating = sentence['label']
  if rating < 3:
    return {"sentiment": 0}
  else:
    return {"sentiment": 1}

In [ ]:
dataset = dataset.map(transform)

In [ ]:
dataset['test'][0]

In [ ]:
#tokenize the dataset
def tokenize_function(example):
    return tokenizer(example['text'], truncation=True)


In [ ]:
tokenized_datasets = dataset.map(tokenize_function, batched=True)
tokenized_datasets

In [ ]:
tokenized_datasets['test'].features
tokenized_datasets['test']['labels']

In [ ]:
from transformers import DataCollatorWithPadding

data_collator = DataCollatorWithPadding(tokenizer=tokenizer)

In [ ]:
tokenized_datasets["train"].column_names

In [ ]:
tokenized_datasets = tokenized_datasets.remove_columns("label")
tokenized_datasets = tokenized_datasets.rename_column("sentiment", "labels")
tokenized_datasets = tokenized_datasets.remove_columns("text")

In [ ]:
from torch.utils.data import DataLoader

train_dataloader = DataLoader(
    tokenized_datasets["train"], shuffle=True, batch_size=8, collate_fn=data_collator
)
eval_dataloader = DataLoader(
    tokenized_datasets["test"], batch_size=8, collate_fn=data_collator
)


In [ ]:
for batch in train_dataloader:
    break
{k: v.shape for k, v in batch.items()}

In [ ]:
tokenized_datasets['train'].features

In [ ]:
from transformers import TrainingArguments

training_args = TrainingArguments("test-trainer")


In [ ]:
model = AutoModelForSequenceClassification.from_pretrained(checkpoint, num_labels=2)


In [ ]:
outputs = model(**batch)
print(outputs.loss, outputs.logits.shape)


In [ ]:
!pip install evaluate


In [ ]:
import evaluate
import numpy as np

accuracy = evaluate.load("accuracy")
f1 = evaluate.load("f1")

def compute_metrics(eval_pred):
    logits, labels = eval_pred
    predictions = np.argmax(logits, axis=-1)

    return {
        "accuracy": accuracy.compute(predictions=predictions, references=labels)["accuracy"],
        "f1_macro": f1.compute(predictions=predictions, references=labels, average="macro")["f1"]
    }

In [ ]:
from torch.optim import AdamW

optimizer = AdamW(model.parameters(), lr=5e-5)

In [ ]:
from transformers import get_scheduler

num_epochs = 3
num_training_steps = num_epochs * len(train_dataloader)
lr_scheduler = get_scheduler(
    "linear",
    optimizer=optimizer,
    num_warmup_steps=0,
    num_training_steps=num_training_steps,
)
print(num_training_steps)

In [ ]:
import torch

device = torch.device("cuda") if torch.cuda.is_available() else torch.device("cpu")
model.to(device)
device

In [ ]:
import evaluate
accuracy = evaluate.load("accuracy")

In [ ]:
from tqdm.auto import tqdm

progress_bar = tqdm(range(num_training_steps))

model.train()
for epoch in range(num_epochs):
    for batch in train_dataloader:
        batch = {k: v.to(device) for k, v in batch.items()}
        outputs = model(**batch)
        loss = outputs.loss
        loss.backward()

        optimizer.step()
        lr_scheduler.step()
        optimizer.zero_grad()
        progress_bar.update(1)
    model.eval()
    for batch in eval_dataloader:
        batch = {k: v.to(device) for k, v in batch.items()}
        with torch.no_grad():
            outputs = model(**batch)

        # Get predicted class
        logits = outputs.logits
        predictions = torch.argmax(logits, dim=-1)

        # Add to metric
        accuracy.add_batch(predictions=predictions, references=batch["labels"])

    eval_result = accuracy.compute()
    print(f"Epoch {epoch+1} eval accuracy: {eval_result['accuracy']:.4f}")
    model.train()  # go back to training mode